In [204]:
import aplpy
import makenicefigure
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii
from astropy.table import Table
from astropy.io import fits
from astropy import wcs
from astropy.io import fits

#read in all the spectral catalogs 
bgpsdata=Table.read("/Users/catherinezucker/Dropbox/Filaments_Comp_Paper/Spectral_Catalogs/BGPS_Final.txt",format='ascii')
atlasgaldata=Table.read("/Users/catherinezucker/Dropbox/Filaments_Comp_Paper/Spectral_Catalogs/ATLASGAL.txt",format='ascii')
hopsdata=Table.read("/Users/catherinezucker/Dropbox/Filaments_Comp_Paper/Spectral_Catalogs/HOPS_Catalog.txt",format='ascii')
malt90data=Table.read("/Users/catherinezucker/Dropbox/Filaments_Comp_Paper/Spectral_Catalogs/MALT90_Catalog.txt",format='ascii')


In [205]:
#Read in our mask file
hdulist = fits.open("/Users/catherinezucker/Dropbox/Filaments_Comp_Paper/Amy/"+fil+"/"+fil+"_Mask_Final.fits")
mask=hdulist[0].data

# Parse the WCS header
w = wcs.WCS(hdulist[0].header)

#Extract relevant BGPS longitude, latitude, and velocity columns from table
#You can extract correct column by printing the table (just type "bgpsdata" etc) 
#and it will indicate what string lon, lat, and vel are called in each text file
BGPS_l=bgpsdata["Glon"] 
BGPS_b=bgpsdata["Glat"]
BGPS_v=bgpsdata["vLSR"]
BGPS_world=np.vstack((BGPS_l,BGPS_b))
BGPS_world=np.array(BGPS_world).T

#Convert longitude and latitude values to pixel coordinates so we can test if in mask
pixcrd = w.wcs_world2pix(BGPS_world, 1)

#Restrict test points to those that fall inside bounds of image mask
inbounds=(pixcrd[:,0] > 0) & (pixcrd[:,0] < mask.shape[1]-1) & (pixcrd[:,1] > 0) & (pixcrd[:,1]<mask.shape[0]-1)

#Now test which points that fall within the image boundaries are within the filament boundaries 
infil=np.where(mask[pixcrd[inbounds][:,1].astype(int), pixcrd[inbounds][:,0].astype(int)]==1)[0]

#Combine the two masks together so we can pick out the points from original BGPS_l, BGPS_v arrays
final=np.where(inbounds==True)[0][infil]

#Here are all the points in the catalog with l,b inside filament mask
lcat=BGPS_l[final]
bcat=BGPS_b[final]
vcat=BGPS_v[final]

#BUT unfortunately not all l,b points have a corresponding velocity measurement, so we'll also have to cut by that too
vreal=np.where(np.isfinite(vcat)==True)

#Here are the final plottable l,v values for your overlays (b is here just for reference)!
lplot=lcat[vreal]
bplot=bcat[vreal]
vplot=vcat[vreal]